**Install Dependencies**

In [1]:
!pip install -q langgraph langchain langchain-openai langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


**Import Libraries**

In [2]:
import os
from langgraph.graph import StateGraph
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from typing import TypedDict, Dict

**Environment Setup**

In [4]:
os.environ["OPENAI_API_KEY"] = "your-api-key"
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

**Shared State**

In [5]:
class State(TypedDict):
    task: str
    code: str
    docs: str

**Code Generator Class**

In [6]:
class CodeGenerator:
    def generate_code(self, state: State) -> Dict:
        response = llm.invoke([HumanMessage(content=f"Write clean Python code for: {state['task']}")])
        return {"code": response.content}

**Documentation Writer Class**

In [7]:
class DocumentationWriter:
    def write_documentation(self, state: State) -> Dict:
        response = llm.invoke([HumanMessage(content=f"Write concise documentation for:\n{state['code']}")])
        return {"docs": response.content}

**Test Writer Class**

In [8]:
class TestWriter:
    def write_tests(self, state: State) -> Dict:
        response = llm.invoke([HumanMessage(content=f"Write unit tests for:\n{state['code']}")])
        return {"docs": state.get("docs", "") + "\n\nTESTS:\n" + response.content}

**Workflow Builder**

In [9]:
def workflow(agent_name: str):
    g = StateGraph(State)
    code_agent, doc_agent, test_agent = CodeGenerator(), DocumentationWriter(), TestWriter()

    if agent_name == "1":
        g.add_node("generate_code", code_agent.generate_code)
        g.set_entry_point("generate_code")

    elif agent_name == "2":
        g.add_node("write_documentation", doc_agent.write_documentation)
        g.set_entry_point("write_documentation")

    elif agent_name == "3":
        g.add_node("write_tests", test_agent.write_tests)
        g.set_entry_point("write_tests")

    elif agent_name == "4":
        g.add_node("generate_code", code_agent.generate_code)
        g.add_node("write_documentation", doc_agent.write_documentation)
        g.add_node("write_tests", test_agent.write_tests)
        g.set_entry_point("generate_code")
        g.add_edge("generate_code", "write_documentation")
        g.add_edge("write_documentation", "write_tests")

    return g.compile()

**User Input**

In [10]:
task = input(" Enter software task: ")

print("\nChoose an agent:")
print("1. Code Generator – writes Python code for your task")
print("2. Documentation Writer – explains the code")
print("3. Test Writer – creates tests for the code")
print("4. Full Pipeline – does all of the above\n")
agent_choice = input("Select option (1-4): ")

 Enter software task: Create a Python program to check if a number is prime

Choose an agent:
1. Code Generator – writes Python code for your task
2. Documentation Writer – explains the code
3. Test Writer – creates tests for the code
4. Full Pipeline – does all of the above

Select option (1-4): 1


**Run Workflow**

In [11]:
state = {"task": task, "code": "", "docs": ""}
out = workflow(agent_choice).invoke(state)

print("\n=== CODE ===\n", out.get("code", ""))
print("\n=== DOCUMENTATION & TESTS ===\n", out.get("docs", ""))


=== CODE ===
 def is_prime(num):
    if num <= 1:
        return False
    for i in range(2, int(num**0.5) + 1):
        if num % i == 0:
            return False
    return True

num = int(input("Enter a number: "))
if is_prime(num):
    print(num, "is a prime number")
else:
    print(num, "is not a prime number")

=== DOCUMENTATION & TESTS ===
 
